In [28]:
# Example of OpenAI function calling

# Key difference from StructuredTool is that 
# - In StructuredTool agent, the inputs come in as K:V dictionary which need to be parsed into InputSchema and then used
# - In OpenAIFunctions agent, the inputs come in as specific fields directly

from langchain.agents import initialize_agent, AgentType
from langchain.chat_models import ChatOpenAI
from langchain.tools import BaseTool, Tool
from langchain.tools.render import format_tool_to_openai_function
from langchain.pydantic_v1 import BaseModel, Field
from typing import Type

class AdditionInputs(BaseModel):
    first_number: int = Field(..., description="The first number")
    second_number: int = Field(..., description="The second number")

class CustomAdditionTool(BaseTool):
    name = "addition_calculator"
    description = "Useful when you want to add two numbers"
    args_schema: Type[BaseModel] = AdditionInputs

    def _run(self, first_number: int, second_number: int):
        return first_number + second_number

    def _arun(self, first_number: int, second_number: int):
        raise NotImplementedError("Not supported")
    
class SubtractionInputs(BaseModel):
    larger_number: int = Field(..., description="The first number")
    smaller_number: int = Field(..., description="The second number")

class CustomSubtractionTool(BaseTool):
    name = "subtraction_calculator"
    description = "Useful when you want to find difference two numbers"
    args_schema: Type[BaseModel] = SubtractionInputs

    def _run(self, larger_number: int, smaller_number: int):
        return larger_number - smaller_number

    def _arun(self, larger_number: int, smaller_number: int):
        raise NotImplementedError("Not supported")
    
tools = [CustomAdditionTool(), CustomSubtractionTool()]    

llm = ChatOpenAI(model="gpt-3.5-turbo-0613")

agent_executor = initialize_agent(llm=llm, tools=tools, agent=AgentType.OPENAI_FUNCTIONS, verbose=True)

In [29]:
agent_executor.invoke({"input": "What is the sum of 50 and 24?"})



> Entering new AgentExecutor chain...

Invoking: `addition_calculator` with `{'first_number': 50, 'second_number': 24}`


74The sum of 50 and 24 is 74.

> Finished chain.


{'input': 'What is the sum of 50 and 24?',
 'output': 'The sum of 50 and 24 is 74.'}

In [30]:
agent_executor.invoke({"input": "What is the difference of 50 and 24?"})



> Entering new AgentExecutor chain...

Invoking: `subtraction_calculator` with `{'larger_number': 50, 'smaller_number': 24}`


26The difference between 50 and 24 is 26.

> Finished chain.


{'input': 'What is the difference of 50 and 24?',
 'output': 'The difference between 50 and 24 is 26.'}

In [31]:
agent_executor.invoke({"input": "What is the difference of 24 and 50?"})



> Entering new AgentExecutor chain...

Invoking: `subtraction_calculator` with `{'larger_number': 50, 'smaller_number': 24}`


26The difference between 24 and 50 is 26.

> Finished chain.


{'input': 'What is the difference of 24 and 50?',
 'output': 'The difference between 24 and 50 is 26.'}